<a href="https://colab.research.google.com/github/CorleoneAppolonia/PMP-2024/blob/main/EXAMEN/subiectul_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Subiectul 2:

In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az


iris = pd.read_csv("iris.csv")
features = ["sepal_length", "sepal_width", "petal_length", "petal_width"]

K = 3
idatas = {}

# 3) Construim, pe rând, câte un model Mixture 1D pentru fiecare feature
for feat in features:
    X_1d = iris[feat].values

    with pm.Model() as model_1d:
        #   (Dirichlet)
        w = pm.Dirichlet("w", a=np.ones(K))

        #  Parametrii Gaussieni (
        mus = pm.Normal("mus", mu=0, sigma=10, shape=K)
        sigmas = pm.HalfNormal("sigmas", sigma=5, shape=K)

        #  Mixture 1D combinand 3 Gaussiene
        comps = [
            pm.Normal.dist(mu=mus[k], sigma=sigmas[k])
            for k in range(K)
        ]

        like = pm.Mixture(
            "like",
            w=w,
            comp_dists=comps,
            observed=X_1d
        )

        #  Sampling -> cerem direct InferenceData cu log_likelihood (necesar pt. WAIC)
        idata_1d = pm.sample(
            2000, tune=1000,
            return_inferencedata=True,
            idata_kwargs={"log_likelihood": True},
            random_seed=42
        )

    idatas[feat] = idata_1d

#  compar toate cele 4 modele folosind WAIC (sau LOO)
comparison_df = az.compare(idatas, method="waic")
print(comparison_df)


Output()

Output()

ERROR:pymc.stats.convergence:There were 7 divergences after tuning. Increase `target_accept` or reparameterize.
ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Output()

Output()

Explicatii

Dirichlet:
w = pm.Dirichlet("w", a=np.ones(K)) genereaza greutatile amestecului, adica ponderea fiecarui cluster in totalul datelor.

Parametrii Gaussieni:
mus si sigmas sunt, pentru fiecare cluster, media si deviatia standard (1D). Am ales priors largi (Normal(0,10), HalfNormal(5)) pentru a nu constrange exagerat.

Mixture:
pm.Mixture("like", w=w, comp_dists=...) creeaza verosimilitatea observata pentru datele X_1d.

Sampling:
pm.sample(2000, tune=1000, return_inferencedata=True, idata_kwargs={"log_likelihood": True}) ne da un obiect InferenceData care contine log-likelihood (necesar pentru WAIC/LOO). Folosim random_seed=42 pentru reproductibilitate.

az.compare:
Produce un DataFrame cu coloane rank, waic, d_waic etc. Modelul cu rank=0 si WAIC minim e considerat cel mai bun (are cea mai buna separare a datelor, raportat la un mixture cu 3 clustere).